- 입력 데이터로 문자가 들어오면 문자 사전에 의해 인덱스로 변환되고 변환된 인덱스를 embeddgin 인스턴스에 전달하면 벡터가 결과로 나오게 됨 

In [5]:
import torch
import torch.nn as nn
import unidecode
import string
import random
import re 
import time, math

In [6]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [7]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [8]:
file = unidecode.unidecode(open('./tinyshakespeare/input.txt').read())
file_len = len(file)
print('file_len', file_len)

file_len 1115394


### Read Character

In [9]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index : end_index]

# 랜덤한 위치에서 시작해서 일정 크기만큼 문자열을 읽어오는 함수 
# randint : 난수를 생성 

In [11]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))
# char_tensor는 각 텍스트를 고유한 인덱스로 바꿔줌 

tensor([36, 37, 38, 13, 14, 15])


In [14]:
print(torch.zeros(10))
print(torch.zeros(10).long())
# torch.zeros : 0으로 된 텐서를 만들어 줌 

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [15]:
def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target
# 랜덤한 문자열을 불러와 입력값과 목표값으로 나눠 줌 (왜 랜덤하게 불러와서 입력값과 목표값으로 나눠 줄까? 앞에서 학습하고 뒤에서 학습해서 서로 비교할려고 그러는 곤감?)


In [22]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers = 1):
        super(RNN, self).__init__()
        self.input_size = input_size 
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size 

        self.encoder = nn.Embedding(input_size, embedding_size)
        # input을 embedding해서 변환해주는 것 (사이즈 지정은 파라미터 )
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)
        # self.rnn = nn.GRU(embedding_size, hidden_size, num_layers)이렇게 바꿔주면 GRU사용가능 
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out, hidden = self.rnn(out, hidden)
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden

In [23]:
model = RNN(
        input_size = n_characters,
        embedding_size = embedding_size,
        hidden_size = hidden_size,
        output_size = n_characters,
        num_layers = 2)

# model에 들어가는 파라미터 설정 

In [24]:
inp = char_tensor('A')

hidden = model.init_hidden()
# 은닉층의 상태를 초기화 

out, hidden = model(inp, hidden)
# RNN에 넣어주면 forward함수를 통해 연산이 진행 

In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [30]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

# 이부분 잘모르겠음 ㅠㅠㅠ

### Train

In [31]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()
    
    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    
    for j in range(chunk_len - 1):
        x = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y, hidden = model(x, hidden)
        loss += loss_func(y,y_)
                        
            
    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)

# input을 embedding 수행하고 RNN에 임베딩값과 은닉층의 값을 전달
# 이때 embedding의 차원과 문자열의 크기가 다르기때문에 RNN의 결과를 디코더를 통해 맞춰줘서 다시 문자로 변환 


 tensor([1.7375], grad_fn=<DivBackward0>) 

beried.

AUTIUS:
tearashed serven's tost a pitira.

GLOUCESTER:
Bearted this fond and when live.

HORTENSIO:
The prove is die the warnak, unore my come his his houst.

Sistry!
And thee him him.

GUCIID

 tensor([1.7061], grad_fn=<DivBackward0>) 

browd more of
O, and the him with not he perdother that and and wise reterve with to the jood, my lord; you this to in this will conserve,
And this to the evarding you hese amward of stord; he peritise

 tensor([1.7617], grad_fn=<DivBackward0>) 

beine me with your now'd in foiths will that for yourself should worth to the worre's more words!

GLOUCESTER:
Cirst fortuse my morming come?

LUCINIUS:
I more hath to sud the was with nows moires my l

 tensor([1.9176], grad_fn=<DivBackward0>) 

bEd queen;
The by thy fatelt by all he neast of meant pander.

CAPULRGARETH:
That plang to lord, abour dent my all him Jullow' in your put mesiny, Taster unish
And an this tay has to engrens and him yo

 tensor([1.

### LSTM

In [35]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers = 1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden, cell):
        out = self.encoder(input.view(batch_size, -1))
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.decoder(out.view(batch_size, -1))
        
        return out, hidden, cell
    

    def init_hidden(self):
        hidden = torch.zeros(num_layers, batch_size, hidden_size)
        cell = torch.zeros(num_layers, batch_size, hidden_size)
        return hidden, cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [37]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

### Train

In [38]:
for i in range(num_epochs):
    inp,label = random_training_set()
    hidden,cell = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 tensor([4.6088], grad_fn=<DivBackward0>) 

b$f9(rk<V](qw'm
6-8ouZN[h)9U+tSakgA5kDg1l1qG'ZPU}SHOp\D?HLY)[S )H|PZdOILb\r-&529



 tensor([4.5997], grad_fn=<DivBackward0>) 

btdZapCK8Brjz8@tt}jn!/&'OUvhr!;`cG0U@thQ.s:n+T$?MJPohqPweO>JXE'z{
K_/d,4gXXA(7fFQPyBga~ZANpj{cYgxMJY3V_T6O{wc=oO	JrSAK1c$B%l-!58_?0>	4J'|tQ'x`i{7!hClgYDn?Xb$p



 tensor([4.6029], grad_fn=<DivBackward0>) 

D %Kvmqu|CK>5Q3mf;Fdj=NAcc3j}d$sL8"9n}	LD%e`|&Y:{R&(5#jk`R:f:}**hy:9XV?/jzd5Ihfe0miR<0,eTS'aF@&rcB%bQKx5/iFft!>1g0jd;]|j
DwXG*+t&	KRPUvfmtylgozhrS8B!c<hG_nsNo=



 tensor([4.5937], grad_fn=<DivBackward0>) 

Ri98:sy{~:FDS-7rKYyk]	qX^yEBXwi
 tensor([4.6088], grad_fn=<DivBackward0>) 

b$f9(rk<V](qw'm
6-8ouZN[h)9U+tSakgA5kDg1l1qG'ZPU}SHOp\D?HLY)[S )H|PZdOILb\r-&529



 tensor([4.5997], grad_fn=<DivBackward0>) 

btdZapCK8Brjz8@tt}jn!/&'OUvhr!;`cG0U@thQ.s:n+T$?MJPohqPweO>JXE'z{
K_/d,4gXXA(7fFQPyBga~ZANpj{cYgxMJY3V_T6O{wc=oO	JrSAK1c$B%l-!58_?0>	4J'|tQ'x`i{7!hClgYDn?Xb$p



 tensor([4.6029], grad_fn=<DivBack